In [2]:
import pandas as pd

data = pd.read_csv("preprocessed_apartment_rentals_one_hot_encoded_states.csv")
print(data["state"])
# alle Spalten droppen, die mit 'state_' anfangen
cols_to_drop = [col for col in data.columns if col.startswith("state_")]
data = data.drop(columns=cols_to_drop)
data.to_csv("preprocessed_apartment_rentals_no_state_dummies_nocluster.csv", index=False)



0        CA
1        VA
2        NC
3        CA
4        NM
         ..
98617    TX
98618    FL
98619    CA
98620    NC
98621    CA
Name: state, Length: 98622, dtype: object
